In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install texthero

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotib inline
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import texthero as hero
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
nltk.download('vader_lexicon')
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def embed_glove(model, s):
    #print(s)
    return_ls = []
    for sentence in s:
        array = []
        for word in str(sentence).split():
            try:
                array.append(model.get_vector(word))
            except:
                pass
        array = np.array(array)
        #print(array.shape)
        return_ls.append(array.mean(axis = 0))
    return return_ls

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/sNLP_project/SongRecommendationChatBot/analysis/word_embedding_model')

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
# Convert
input_file = 'new_glove.6B.100d.txt'
output_file = 'gensim_glove.6B.100d.txt'
glove2word2vec(input_file, output_file)

# Test Glove model
model = KeyedVectors.load_word2vec_format(output_file, binary=False)
word = 'cat'
print(word)
print('Most similar:\n{}'.format(model.most_similar(word)))

cat
Most similar:
[('dog', 0.8798074722290039), ('rabbit', 0.7424426674842834), ('cats', 0.7323004007339478), ('monkey', 0.7288709878921509), ('pet', 0.7190139889717102), ('dogs', 0.7163872718811035), ('mouse', 0.6915250420570374), ('puppy', 0.6800068020820618), ('rat', 0.6641027331352234), ('spider', 0.6501135230064392)]


In [ ]:
def setimentanalyzer(df):
    neg='Negative'
    neu='Neutral'
    pos='Positive'
    negative = []
    neutral = []
    positive = []
    dominant_sentiment=[]
    dominant_sentiment_score=[]
    #Initialize the model
    sid = SentimentIntensityAnalyzer()
    #Iterate for each row of lyrics and append the scores
    for i in df.index:
       
        scores = sid.polarity_scores(df['Lyric'].iloc[i])
        negative.append(scores['neg'])
        neutral.append(scores['neu'])
        positive.append(scores['pos'])
        if scores['neg']>scores['pos']:
            dominant_sentiment_score.append(scores['neg'])
            dominant_sentiment.append(neg)
        elif scores['neg']<scores['pos']:
            dominant_sentiment_score.append(scores['pos'])
            dominant_sentiment.append(pos)
        else:
            dominant_sentiment_score.append(scores['neu'])
            dominant_sentiment.append(neu)
    #Create 5 columns to the main data frame  for each score
    df['negative'] = negative
    df['neutral'] = neutral
    df['positive'] = positive
    df['dominant_sentiment']=dominant_sentiment
    df['dominant_sentiment_score']=dominant_sentiment_score
    return df

In [ ]:
#load the data
df = pd.read_csv('../data/TaylorSwift.csv')
df['embedded'] = embed_glove(model, df['Lyric'])
df = df.dropna()
df = df.reset_index()
df.head()

In [ ]:
#album name
album_name = list(set(df.Album))
album_num = []
for i in range(df.shape[0]):
  album = str(df.Album[i])
  album_num.append(album_name.index(album))
df['Album_num'] = album_num
df.head()

tsne by album

In [ ]:
#embedding
embedding = np.stack([i for i in df.embedded],axis=0)
df_embed = pd.concat([df,pd.DataFrame(embedding)],axis=1)
#tsne
tsne = TSNE(n_components=3)
df_tsne1 = tsne.fit_transform(df_embed.iloc[:,9:])
df_tsne1 = pd.DataFrame(df_tsne1,
                            columns =["Component1","Component2","Component3"])
df_tsne1.to_csv('../data/df_tsne1.csv')

tsne by sentiment

In [ ]:
#df sentiment
df_sent = setimentanalyzer(df)
df_sent = df_sent[['dominant_sentiment','negative','neutral', 'positive']]
df_sent = pd.concat([df_sent,pd.DataFrame(embedding)],axis=1)

#sentiment label
sent_name = list(set(df_sent.dominant_sentiment))
sent_num = []
for i in range(df_sent.shape[0]):
  sent = str(df.dominant_sentiment[i])
  sent_num.append(sent_name.index(sent))
df_sent['sent_num'] = sent_num

#tsne
tsne = TSNE(n_components=3)
df_tsne_s = tsne.fit_transform(df_sent.iloc[:,1:])
df_tsne_s = pd.DataFrame(df_tsne_s,
                            columns =["C1","C2","C3"])
df_tsne_s = pd.concat([df_tsne_s,df_sent['sent_num']],axis=1)
df_tsne_s.to_csv('../data/df_tsne_s.csv')

plotly 3d tsne visualization

In [ ]:
#data load
df_tsne_s = pd.read_csv('../data/df_tsne_s.csv',index_col=0,header=0)
df_tsne1 = pd.read_csv('../data/df_tsne1.csv',index_col=0,header=0)

In [ ]:
#tsne by sentiment
import plotly.express as px

fig_3d = px.scatter_3d(
    df_tsne_s, x='C1', y='C2', z='C3',
    color=df_tsne_s.sent_num, labels={'color': 'sentiment'}
)
fig_3d.update_traces(marker_size=5)
fig_3d.update_layout(title={'text': "T-SNE plot by sentiment",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'})
fig_3d.show()
fig_3d.write_image("../3d_sentiment.png")

In [ ]:
#tsne by album
fig_3d = px.scatter_3d(
    df_tsne1, x='Component1', y='Component2', z='Component3',
    color=df_tsne1.Album_num, labels={'color': 'Album'}
)
fig_3d.update_traces(marker_size=5)
fig_3d.update_layout(title={'text': "T-SNE plot by album",'y':0.95,'x':0.5,'xanchor': 'center','yanchor': 'top'})
fig_3d.show()